## 1.Monte um passo a passo para o Bagging

<div style="background-color: #e6f7ff; padding: 20px; border: 3px solid #00b3b3; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">
   <strong>Resposta:</strong>
    Primeiramente se cria  um n número de novos datasets como a mesma quantidade de linhas, podendo haver reposição de linhas esse  passo é chamado de Bootstrap, para cada subconjunto de dados gerado, treine cada modelo separado podendo optar por exemplo por pelo algoritmo de árvore de decisão, ou KNN, ou regressão linear , após combine as predições dos modelos para obter a predição final essa pode ser feita de duas formas para classificação binária se faz por quantidade majoritária de votos ou em caso de uma regressão se faz uma média e esse passo é chamado de Aggregating. 

</div>


## 2.Explique com suas palavras o Bagging

<div style="background-color: #e6f7ff; padding: 20px; border: 3px solid #00b3b3; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">
   <strong>Resposta:</strong>
     Bootstrap aggregating, também chamado de Bagging, é um algoritmo de aprendizado de máquina projetado para melhorar a estabilidade e a precisão que tem como principal função evitar o overfitting.

</div>

## 3. (Opcional) Implementar em python o código do Bagging

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import warnings
warnings.filterwarnings('ignore')

In [13]:
df =sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [15]:
features = df[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
target = df['survived']

#tratamento
features.fillna({'age':features['age'].median()}, inplace=True)
features.embarked = features.embarked.map({'C': 0, 'S':1, 'Q':2})
features.fillna({'embarked':features['embarked'].mode()[0]}, inplace=True)
features.sex = features.sex.map({'female': 1, 'male':0})

features

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,0,22.0,1,0,7.2500,1.0
1,1,1,38.0,1,0,71.2833,0.0
2,3,1,26.0,0,0,7.9250,1.0
3,1,1,35.0,1,0,53.1000,1.0
4,3,0,35.0,0,0,8.0500,1.0
...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,1.0
887,1,1,19.0,0,0,30.0000,1.0
888,3,1,28.0,1,2,23.4500,1.0
889,1,0,26.0,0,0,30.0000,0.0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=41513)

n_estimators = 20 
classifiers = []

# bootstrap
for _ in range(n_estimators):
    # Criar uma amostra bootstrap
    X_bootstrap, y_bootstrap = resample(X_train, y_train, random_state=_)
    # Treinar o classificador base
    classifier = DecisionTreeClassifier()
    classifier.fit(X_bootstrap, y_bootstrap)
    classifiers.append(classifier)

# Prevendo os resultados para cada classificador
predicoes = np.zeros((X_test.shape[0], len(classifiers)))

# Para cada classificador, obtemos as previsões
for i, clf in enumerate(classifiers):
  predicoes[:, i] = clf.predict(X_test)

# Votação majoritária: escolhemos a classe que foi mais votada
from scipy.stats import mode
y_pred_final = mode(predicoes, axis=1)[0].ravel()

# Avaliar o desempenho final
accuracy = accuracy_score(y_test, y_pred_final)
print(f'Acurácia final: {accuracy:.4f}')

Acurácia final: 0.7989
